https://python.langchain.com/v0.1/docs/use_cases/question_answering/quickstart/

In [4]:
# import openai
# from langchain.chains import ConversationalRetrievalChain, RetrievalQA
# from langchain.chat_models import ChatOpenAI
# from langchain.document_loaders import DirectoryLoader, TextLoader
# from langchain_openai import OpenAIEmbeddings
# from langchain.indexes import VectorstoreIndexCreator
# from langchain.indexes.vectorstore import VectorStoreIndexWrapper
# from langchain.llms import OpenAI
# from langchain.vectorstores import Chroma


import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
# from langchain_chroma import Chroma
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [13]:
import os
import sys
sys.path.append('../..')

from py3810.myUtils import pickle_dump, pickle_load
path_lumen_docs = '..\langchain\docs\lumen\\docs\\'
path_lumen_docs_chatbot = '..\langchain\docs\lumen\\docs\\chatbot\\'

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('.env\my_api_key.env')) # read local .env file

os.environ["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"]
os.environ["SECRET_KEY"]


'my_secret_key'

In [11]:
import textwrap

def print_wrapped(text, width=80):
  """
  Prints a long string to the console, wrapped to fit within a specified width.

  Args:
      text: The long string to be wrapped.
      width: The desired width for each line (default: 80 columns).
  """
  wrapped_text = textwrap.wrap(text, width=width)
  for line in wrapped_text:
    print(line)

# Example usage
long_string = "This is a very long string that needs to be wrapped to fit within 80 columns. It can contain spaces, punctuation, and even newlines."
print_wrapped(long_string)

This is a very long string that needs to be wrapped to fit within 80 columns. It
can contain spaces, punctuation, and even newlines.


In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [14]:
import os

def get_file_names(directory):
    return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

print(get_file_names(path_lumen_docs_chatbot))

['lumen_docs_processed_pdfs', 'lumen_docs_processed_videos', 'lumen_docs_processed_website', 'lumen_docs_processed_youtube']


In [ ]:
docs = pickle_load(filename_pickle='lumen_website_docs_processed', path_pickle_dump=path_lumen_docs)

In [ ]:
docs

In [ ]:
query = 'where is lumen optometric?'

persist_directory = "./chroma_db"
embedding_function = OpenAIEmbeddings()
collection_name = 'lumen_website_docs'

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
# vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

#### Only run cell below to add new data


### References Chroma db  
https://docs.trychroma.com/guides  
https://github.com/neo-con/chromadb-tutorial  
https://python.langchain.com/v0.1/docs/integrations/vectorstores/chroma/  


In [ ]:

# # save to disk
# db2 = Chroma.from_documents(
#   documents=splits,
#   embedding=embedding_function,
#   collection_name=collection_name,
#   persist_directory=persist_directory
#   )
# db2_ans = db2.similarity_search(query)
# # print(docs[0].page_content)
# print(f'db2_ans:{db2_ans}')

In [ ]:
# load from disk

db3 = Chroma(
  embedding_function=embedding_function,
  collection_name=collection_name,
  persist_directory=persist_directory
  )
db3_ans = db3.similarity_search(query)
print(f'db3_ans:{db3_ans}')

In [ ]:
vectorstore = Chroma(
  embedding_function=embedding_function,
  collection_name=collection_name,
  persist_directory=persist_directory
  )

In [ ]:
# # Load, chunk and index the contents of the blog.
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)
# vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
# retriever = vectorstore.as_retriever()
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
print_wrapped(rag_chain.invoke("What is lumen optometric's phone number?"))

In [ ]:
print_wrapped(rag_chain.invoke("What is lumen optometric's address?"))

In [ ]:
retriever.get_relevant_documents("What is lumen optometric's location?")

In [ ]:
print_wrapped(rag_chain.invoke("What is lumen optometric's location?"))

In [ ]:
print_wrapped(rag_chain.invoke("do you take vision insurance"))

In [ ]:
print_wrapped(rag_chain.invoke("What is lumen optometric's location?"))

In [ ]:
print_wrapped(rag_chain.invoke("What is lumen optometric's address?"))

In [ ]:
print_wrapped(rag_chain.invoke("What is Ortho-k"))

In [ ]:
print_wrapped(rag_chain.invoke("What equipment do they have?"))